<a href="https://colab.research.google.com/github/Ragulan-K/FYP/blob/main/Final_year_project_using_XGBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 11.3 MB/s eta 0:00:00


In [9]:
import numpy as np
import pandas as pd
import re

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import optuna
import xgboost as xgb

from sklearn.metrics import r2_score

In [4]:
data=pd.read_csv('/content/SolarPrediction.csv')

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32686 entries, 0 to 32685
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   UNIXTime                32686 non-null  int64  
 1   Data                    32686 non-null  object 
 2   Time                    32686 non-null  object 
 3   Radiation               32686 non-null  float64
 4   Temperature             32686 non-null  int64  
 5   Pressure                32686 non-null  float64
 6   Humidity                32686 non-null  int64  
 7   WindDirection(Degrees)  32686 non-null  float64
 8   Speed                   32686 non-null  float64
 9   TimeSunRise             32686 non-null  object 
 10  TimeSunSet              32686 non-null  object 
dtypes: float64(4), int64(3), object(4)
memory usage: 2.7+ MB


In [7]:
print("Total missing values:",data.isna().sum().sum())

Total missing values: 0


##Feature Engineering

In [8]:
data

,UNIXTime,Data,Time,Radiation,Temperature,Pressure,Humidity,WindDirection(Degrees),Speed,TimeSunRise,TimeSunSet
0,1475229326,9/29/2016 12:00:00 AM,23:55:26,1.21,48,30.46,59,177.39,5.62,06:13:00,18:13:00
1,1475229023,9/29/2016 12:00:00 AM,23:50:23,1.21,48,30.46,58,176.78,3.37,06:13:00,18:13:00
2,1475228726,9/29/2016 12:00:00 AM,23:45:26,1.23,48,30.46,57,158.75,3.37,06:13:00,18:13:00
3,1475228421,9/29/2016 12:00:00 AM,23:40:21,1.21,48,30.46,60,137.71,3.37,06:13:00,18:13:00
4,1475228124,9/29/2016 12:00:00 AM,23:35:24,1.17,48,30.46,62,104.95,5.62,06:13:00,18:13:00
...,...,...,...,...,...,...,...,...,...,...,...
32681,1480587604,12/1/2016 12:00:00 AM,00:20:04,1.22,44,30.43,102,145.42,6.75,06:41:00,17:42:00
32682,1480587301,12/1/2016 12:00:00 AM,00:15:01,1.17,44,30.42,102,117.78,6.75,06:41:00,17:42:00
32683,1480587001,12/1/2016 12:00:00 AM,00:10:01,1.20,44,30.42,102,145.19,9.00,06:41:00,17:42:00
32684,1480586702,12/1/2016 12:00:00 AM,00:05:02,1.23,44,30.42,101,164.19,7.87,06:41:00,17:42:00


In [15]:
data['Month'] = data['Data'].apply(lambda x: re.search(r'^\d+', x).group(0)).astype(int)
data['Day'] = data['Data'].apply(lambda x: re.search(r'(?<=\/)\d+(?=\/)', x).group(0)).astype(int)
data['Year'] = data['Data'].apply(lambda x: re.search(r'(?<=\/)\d+(?=\s)', x).group(0)).astype(int)

data = data.drop('Data', axis=1)

In [16]:
data

,UNIXTime,Time,Radiation,Temperature,Pressure,Humidity,WindDirection(Degrees),Speed,TimeSunRise,TimeSunSet,Month,Day,Year
0,1475229326,23:55:26,1.21,48,30.46,59,177.39,5.62,06:13:00,18:13:00,9,29,2016
1,1475229023,23:50:23,1.21,48,30.46,58,176.78,3.37,06:13:00,18:13:00,9,29,2016
2,1475228726,23:45:26,1.23,48,30.46,57,158.75,3.37,06:13:00,18:13:00,9,29,2016
3,1475228421,23:40:21,1.21,48,30.46,60,137.71,3.37,06:13:00,18:13:00,9,29,2016
4,1475228124,23:35:24,1.17,48,30.46,62,104.95,5.62,06:13:00,18:13:00,9,29,2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...
32681,1480587604,00:20:04,1.22,44,30.43,102,145.42,6.75,06:41:00,17:42:00,12,1,2016
32682,1480587301,00:15:01,1.17,44,30.42,102,117.78,6.75,06:41:00,17:42:00,12,1,2016
32683,1480587001,00:10:01,1.20,44,30.42,102,145.19,9.00,06:41:00,17:42:00,12,1,2016
32684,1480586702,00:05:02,1.23,44,30.42,101,164.19,7.87,06:41:00,17:42:00,12,1,2016


In [17]:
data['Hour'] = data['Time'].apply(lambda x: re.search(r'^\d+', x).group(0)).astype(int)
data['Minute'] = data['Time'].apply(lambda x: re.search(r'(?<=:)\d+(?=:)', x).group(0)).astype(int)
data['Second'] = data['Time'].apply(lambda x: re.search(r'\d+$', x).group(0)).astype(int)

data = data.drop('Time', axis=1)

In [18]:
data

,UNIXTime,Radiation,Temperature,Pressure,Humidity,WindDirection(Degrees),Speed,TimeSunRise,TimeSunSet,Month,Day,Year,Hour,Minute,Second
0,1475229326,1.21,48,30.46,59,177.39,5.62,06:13:00,18:13:00,9,29,2016,23,55,26
1,1475229023,1.21,48,30.46,58,176.78,3.37,06:13:00,18:13:00,9,29,2016,23,50,23
2,1475228726,1.23,48,30.46,57,158.75,3.37,06:13:00,18:13:00,9,29,2016,23,45,26
3,1475228421,1.21,48,30.46,60,137.71,3.37,06:13:00,18:13:00,9,29,2016,23,40,21
4,1475228124,1.17,48,30.46,62,104.95,5.62,06:13:00,18:13:00,9,29,2016,23,35,24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32681,1480587604,1.22,44,30.43,102,145.42,6.75,06:41:00,17:42:00,12,1,2016,0,20,4
32682,1480587301,1.17,44,30.42,102,117.78,6.75,06:41:00,17:42:00,12,1,2016,0,15,1
32683,1480587001,1.20,44,30.42,102,145.19,9.00,06:41:00,17:42:00,12,1,2016,0,10,1
32684,1480586702,1.23,44,30.42,101,164.19,7.87,06:41:00,17:42:00,12,1,2016,0,5,2


In [20]:
data['SunriseHour'] = data['TimeSunRise'].apply(lambda x: re.search(r'^\d+', x).group(0)).astype(int)
data['SunriseMinute'] = data['TimeSunRise'].apply(lambda x: re.search(r'(?<=:)\d+(?=:)', x).group(0)).astype(int)

data['SunsetHour'] = data['TimeSunSet'].apply(lambda x: re.search(r'^\d+', x).group(0)).astype(int)
data['SunsetMinute'] = data['TimeSunSet'].apply(lambda x: re.search(r'(?<=:)\d+(?=:)', x).group(0)).astype(int)

data = data.drop(['TimeSunRise', 'TimeSunSet'], axis=1)

In [21]:
data

,UNIXTime,Radiation,Temperature,Pressure,Humidity,WindDirection(Degrees),Speed,Month,Day,Year,Hour,Minute,Second,SunriseHour,SunriseMinute,SunsetHour,SunsetMinute
0,1475229326,1.21,48,30.46,59,177.39,5.62,9,29,2016,23,55,26,6,13,18,13
1,1475229023,1.21,48,30.46,58,176.78,3.37,9,29,2016,23,50,23,6,13,18,13
2,1475228726,1.23,48,30.46,57,158.75,3.37,9,29,2016,23,45,26,6,13,18,13
3,1475228421,1.21,48,30.46,60,137.71,3.37,9,29,2016,23,40,21,6,13,18,13
4,1475228124,1.17,48,30.46,62,104.95,5.62,9,29,2016,23,35,24,6,13,18,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32681,1480587604,1.22,44,30.43,102,145.42,6.75,12,1,2016,0,20,4,6,41,17,42
32682,1480587301,1.17,44,30.42,102,117.78,6.75,12,1,2016,0,15,1,6,41,17,42
32683,1480587001,1.20,44,30.42,102,145.19,9.00,12,1,2016,0,10,1,6,41,17,42
32684,1480586702,1.23,44,30.42,101,164.19,7.87,12,1,2016,0,5,2,6,41,17,42


In [22]:
data.dtypes

UNIXTime                    int64
Radiation                 float64
Temperature                 int64
Pressure                  float64
Humidity                    int64
WindDirection(Degrees)    float64
Speed                     float64
Month                       int64
Day                         int64
Year                        int64
Hour                        int64
Minute                      int64
Second                      int64
SunriseHour                 int64
SunriseMinute               int64
SunsetHour                  int64
SunsetMinute                int64
dtype: object

In [23]:
data['Year'].unique()

array([2016])

In [24]:
data['SunriseHour'].unique()

array([6])

In [25]:
data = data.drop(['Year', 'SunriseHour'], axis=1)

##Splitting & Scaling

In [26]:
y = data['Radiation'].copy()
X = data.drop('Radiation', axis=1).copy()

In [27]:
scaler = StandardScaler()

X = scaler.fit_transform(X)

In [30]:
pd.DataFrame(X)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,-0.937753,-0.500439,0.678974,-0.616253,0.407620,-0.178738,-1.391540,1.512248,1.655482,1.589236,0.660811,-0.943425,1.369126,-1.525765
1,-0.937854,-0.500439,0.678974,-0.654730,0.400285,-0.823359,-1.391540,1.512248,1.655482,1.299687,0.429167,-0.943425,1.369126,-1.525765
2,-0.937953,-0.500439,0.678974,-0.693206,0.183490,-0.823359,-1.391540,1.512248,1.655482,1.010138,0.660811,-0.943425,1.369126,-1.525765
3,-0.938054,-0.500439,0.678974,-0.577776,-0.069497,-0.823359,-1.391540,1.512248,1.655482,0.720589,0.274737,-0.943425,1.369126,-1.525765
4,-0.938153,-0.500439,0.678974,-0.500823,-0.463407,-0.178738,-1.391540,1.512248,1.655482,0.431040,0.506381,-0.943425,1.369126,-1.525765
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32681,0.845373,-1.145490,0.130250,1.038241,0.023209,0.145006,1.344003,-1.701824,-1.672098,-0.437606,-1.037912,0.860876,-0.730393,0.293766
32682,0.845273,-1.145490,-0.052658,1.038241,-0.309138,0.145006,1.344003,-1.701824,-1.672098,-0.727155,-1.269556,0.860876,-0.730393,0.293766
32683,0.845173,-1.145490,-0.052658,1.038241,0.020443,0.789627,1.344003,-1.701824,-1.672098,-1.016704,-1.269556,0.860876,-0.730393,0.293766
32684,0.845073,-1.145490,-0.052658,0.999764,0.248901,0.465884,1.344003,-1.701824,-1.672098,-1.306253,-1.192341,0.860876,-0.730393,0.293766


In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=100)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, train_size=0.8, random_state=200)

In [31]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dval = xgb.DMatrix(X_val, label=y_val)
dtest = xgb.DMatrix(X_test, label=y_test)

In [41]:
def get_model_rmse(params):
    model = xgb.train(params, dtrain, num_boost_round=100, evals=[(dval, 'eval')], early_stopping_rounds=10, verbose_eval=0)
    results = model.eval(dval)
    rmse = float(re.search(r'[\d.]+$', results).group(0))
    return rmse

In [42]:
def objective(trial):
    learning_rate = trial.suggest_loguniform('learning_rate', 0.00001, 10.0)
    max_depth = trial.suggest_int('max_depth', 4, 8)
    l1_reg = trial.suggest_loguniform('l1_reg', 0.00001, 10.0)
    l2_reg = trial.suggest_loguniform('l2_reg', 0.00001, 10.0)

    params = {'learning_rate': learning_rate, 'max_depth': max_depth, 'alpha': l1_reg, 'lambda': l2_reg}

    return get_model_rmse(params)

In [43]:
import optuna

study = optuna.create_study()
study.optimize(objective, n_trials=100, show_progress_bar=True)

[I 2024-03-04 15:50:45,217] A new study created in memory with name: no-name-55f22c2e-d53c-4c35-89e2-3d29baf1fc2f


  0%|          | 0/100 [00:00<?, ?it/s]

<ipython-input-42-0ed627bc5bda>:2: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 0.00001, 10.0)
<ipython-input-42-0ed627bc5bda>:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  l1_reg = trial.suggest_loguniform('l1_reg', 0.00001, 10.0)
<ipython-input-42-0ed627bc5bda>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  l2_reg = trial.suggest_loguniform('l2_reg', 0.00001, 10.0)


[I 2024-03-04 15:50:45,846] Trial 0 finished with value: 185.8458360346366 and parameters: {'learning_rate': 0.007019897550866232, 'max_depth': 7, 'l1_reg': 0.00016987061289884135, 'l2_reg': 0.0003286720153446181}. Best is trial 0 with value: 185.8458360346366.
[I 2024-03-04 15:50:46,070] Trial 1 finished with value: 280.28131943858824 and parameters: {'learning_rate': 0.0017404455919664108, 'max_depth': 4, 'l1_reg': 8.958317905541408e-05, 'l2_reg': 0.058376728523940734}. Best is trial 0 with value: 185.8458360346366.
[I 2024-03-04 15:50:46,141] Trial 2 finished with value: 104.7058948229656 and parameters: {'learning_rate': 1.0748013483235221, 'max_depth': 6, 'l1_reg': 0.1573268010037336, 'l2_reg': 0.058065511765123416}. Best is trial 2 with value: 104.7058948229656.
[I 2024-03-04 15:50:46,389] Trial 3 finished with value: 319.0076765470193 and parameters: {'learning_rate': 1.767261226898824e-05, 'max_depth': 4, 'l1_reg': 1.0002130411314355e-05, 'l2_reg': 6.149510262932192e-05}. Best 

In [44]:
best_params = study.best_params
best_params

{'learning_rate': 0.06968861571466586,
 'max_depth': 8,
 'l1_reg': 0.010244995317540678,
 'l2_reg': 7.992013487604614e-05}

In [45]:
model = xgb.train(best_params, dtrain, num_boost_round=10000, evals=[(dval, 'eval')], early_stopping_rounds=10)

[0]	eval-rmse:300.13137
[1]	eval-rmse:282.26882
[2]	eval-rmse:265.81127
[3]	eval-rmse:250.69038
[4]	eval-rmse:236.62781
[5]	eval-rmse:223.67936
[6]	eval-rmse:211.69449
[7]	eval-rmse:200.51588
[8]	eval-rmse:190.51401
[9]	eval-rmse:181.32111
[10]	eval-rmse:172.98990
[11]	eval-rmse:165.14935
[12]	eval-rmse:158.14825
[13]	eval-rmse:151.66945


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [15:54:01] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "l1_reg", "l2_reg" } are not used.

  warnings.warn(smsg, UserWarning)


[14]	eval-rmse:145.79488
[15]	eval-rmse:140.52176
[16]	eval-rmse:135.54595
[17]	eval-rmse:131.03077
[18]	eval-rmse:126.93109
[19]	eval-rmse:123.21754
[20]	eval-rmse:119.67611
[21]	eval-rmse:116.63519
[22]	eval-rmse:113.98752
[23]	eval-rmse:111.43256
[24]	eval-rmse:109.12833
[25]	eval-rmse:107.01635
[26]	eval-rmse:105.19458
[27]	eval-rmse:103.62183
[28]	eval-rmse:102.20101
[29]	eval-rmse:100.83009
[30]	eval-rmse:99.53392
[31]	eval-rmse:98.45718
[32]	eval-rmse:97.35530
[33]	eval-rmse:96.51904
[34]	eval-rmse:95.55154
[35]	eval-rmse:94.84315
[36]	eval-rmse:94.21890
[37]	eval-rmse:93.63553
[38]	eval-rmse:93.02548
[39]	eval-rmse:92.54065
[40]	eval-rmse:92.08513
[41]	eval-rmse:91.62825
[42]	eval-rmse:91.34045
[43]	eval-rmse:90.92142
[44]	eval-rmse:90.57141
[45]	eval-rmse:90.23959
[46]	eval-rmse:89.99340
[47]	eval-rmse:89.74934
[48]	eval-rmse:89.39709
[49]	eval-rmse:89.20542
[50]	eval-rmse:89.01725
[51]	eval-rmse:88.90058
[52]	eval-rmse:88.72410
[53]	eval-rmse:88.45427
[54]	eval-rmse:88.46772


##Results

In [47]:
y_true = np.array(y_test, dtype=float)
y_pred = np.array(model.predict(dtest), dtype=float)

In [48]:
r2 = r2_score(y_true, y_pred)

print("R^2 Score: {:.4f}".format(r2))

R^2 Score: 0.9403
